## 介绍

* https://zhuanlan.zhihu.com/p/489995404#

In [25]:
import torch
import torchvision

print(f'torch: {torch.__version__} \n'
      f'torchvision: {torchvision.__version__}')

torch: 1.10.1 
torchvision: 0.11.2


## 定义简单的浮点模块


In [2]:
# 定义简单的浮点模块
from torch import nn, Tensor


class M(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = torch.nn.Conv2d(1, 1, 1)
        self.relu = torch.nn.ReLU()

    def _forward_impl(self, x: Tensor) -> Tensor:
        '''提供便捷函数'''
        x = self.conv(x)
        x = self.relu(x)
        return x

    def forward(self, x: Tensor) -> Tensor:
        x= self._forward_impl(x)
        return x

## 定义可量化模块
将浮点模块 `M` 转换为可量化模块 `QM`（量化流程的最关键的一步）。

In [5]:
# from torch.ao.quantization import QuantStub, DeQuantStub
from torch.quantization import QuantStub, DeQuantStub


class QM(M):
    '''
    Args:
        is_print: 为了测试需求，打印一些信息
    '''
    def __init__(self, is_print: bool=False):
        super().__init__()
        self.is_print = is_print
        self.quant = QuantStub() # 将张量从浮点转换为量化
        self.dequant = DeQuantStub() # 将张量从量化转换为浮点

    def forward(self, x: Tensor) -> Tensor:
        # 手动指定张量将在量化模型中从浮点模块转换为量化模块的位置
        x = self.quant(x)
        if self.is_print:
            print('量化前的类型：', x.dtype)
        x = self._forward_impl(x)
        if self.is_print:
            print('量化中的类型：',x.dtype)
        # 在量化模型中手动指定张量从量化到浮点的转换位置
        x = self.dequant(x)
        if self.is_print:
            print('量化后的类型：', x.dtype)
        return x

In [9]:
# 简单测试前向过程的激活数据类型：
input_fp32 = torch.randn(4, 1, 4, 4) # 输入的数据

m = QM(is_print=True)
x = m(input_fp32)

# 查看权重的数据类型：可以看出，此时模块 m 是浮点模块。
m.conv.weight.dtype # torch.float32

量化前的类型： torch.float32
量化中的类型： torch.float32
量化后的类型： torch.float32


torch.float32

## PTQ 简介
当内存带宽和计算空间都很重要时，通常会使用训练后量化，而 CNN 就是其典型的用例。训练后量化对模型的 `权重` 和 `激活` 进行量化。它在可能的情况下将 激活 融合到前面的层中。它需要用具有代表性的数据集进行 校准，以确定激活的最佳量化参数。

## 静态量化模型
直接创建浮点模块的实例：

In [10]:
# 创建浮点模型实例
model_fp32 = QM(is_print=True)

In [11]:
# 要使 PTQ 生效，必须将模型设置为 eval 模式：
model_fp32.eval()

QM(
  (conv): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
  (relu): ReLU()
  (quant): QuantStub()
  (dequant): DeQuantStub()
)

In [12]:
# 查看此时的数据类型：

input_fp32 = torch.randn(4, 1, 4, 4)

x = model_fp32(input_fp32)
print('激活和权重的数据类型分别为：'
      f'{x.dtype}, {model_fp32.conv.weight.dtype}')

量化前的类型： torch.float32
量化中的类型： torch.float32
量化后的类型： torch.float32
激活和权重的数据类型分别为：torch.float32, torch.float32


# 定义观测器qconfig
赋值实例变量 `qconfig`，其中包含关于要附加哪种观测器的信息： - 使用 [`'fbgemm'`](https://github.com/pytorch/FBGEMM) 用于带 AVX2 的 x86（没有AVX2，一些运算的实现效率很低）；使用 [`'qnnpack'`](https://github.com/pytorch/pytorch/tree/master/aten/src/ATen/native/quantized/cpu/qnnpack) 用于 ARM CPU（通常出现在移动/嵌入式设备中）。 
- 其他量化配置，如选择对称或非对称量化和 `MinMax` 或 `L2Norm` 校准技术，可以在这里指定。

In [15]:
model_fp32.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# 查看此时的数据类型：
input_fp32 = torch.randn(4, 1, 4, 4)

x = model_fp32(input_fp32)
print('激活和权重的数据类型分别为：'
      f'{x.dtype}, {model_fp32.conv.weight.dtype}')

量化前的类型： torch.float32
量化中的类型： torch.float32
量化后的类型： torch.float32
激活和权重的数据类型分别为：torch.float32, torch.float32


## 融合激活层

在适用的地方，融合 activation 到前面的层（这需要根据模型架构手动完成）。常见的融合包括 `conv + relu` 和 `conv + batchnorm + relu`。

In [16]:

model_fp32_fused = torch.quantization.fuse_modules(model_fp32,
                                                      [['conv', 'relu']])

model_fp32_fused
# QM(
#   (conv): ConvReLU2d(
#     (0): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
#     (1): ReLU()
#   )
#   (relu): Identity()
#   (quant): QuantStub()
#   (dequant): DeQuantStub()
# )

QM(
  (conv): ConvReLU2d(
    (0): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
  )
  (relu): Identity()
  (quant): QuantStub()
  (dequant): DeQuantStub()
)

可以看到 model_fp32_fused 中 ConvReLU2d 融合 model_fp32 的两个层 conv 和 relu。

查看此时的数据类型：

In [17]:
input_fp32 = torch.randn(4, 1, 4, 4)

x = model_fp32_fused(input_fp32)
print('激活和权重的数据类型分别为：'
      f'{x.dtype}, {model_fp32.conv.weight.dtype}')

量化前的类型： torch.float32
量化中的类型： torch.float32
量化后的类型： torch.float32
激活和权重的数据类型分别为：torch.float32, torch.float32


## 启用观测器
在融合后的模块中启用观测器，用于在校准期间观测激活（activation）张量。
## 校准准备好的模型
校准准备好的模型，以确定量化参数的激活在现实世界的设置，校准具有代表性的数据集。

In [20]:
model_fp32_prepared = torch.quantization.prepare(model_fp32_fused)

# 校准准备好的模型
input_fp32 = torch.randn(4, 1, 4, 4)

x = model_fp32_prepared(input_fp32)
print('激活和权重的数据类型分别为：'
      f'{x.dtype}, {model_fp32.conv.weight.dtype}')
# 量化前的类型： torch.float32
# 量化中的类型： torch.float32
# 量化后的类型： torch.float32
# 激活和权重的数据类型分别为：torch.float32, torch.float32

量化前的类型： torch.float32
量化中的类型： torch.float32
量化后的类型： torch.float32
激活和权重的数据类型分别为：torch.float32, torch.float32


## 模型转换convert
量化权重，计算和存储每个激活张量要使用的尺度（scale）和偏差（bias）值，并用量化实现替换关键算子。

转换已校准好的模型为量化模型：

In [21]:
model_int8 = torch.quantization.convert(model_fp32_prepared)
model_int8

/home/chongqinghuang/anaconda3/envs/waymo_38/lib/python3.8/site-packages/torch/ao/quantization/observer.py:886: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  src_bin_begin // dst_bin_width, 0, self.dst_nbins - 1
/home/chongqinghuang/anaconda3/envs/waymo_38/lib/python3.8/site-packages/torch/ao/quantization/observer.py:891: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.di

QM(
  (conv): QuantizedConvReLU2d(1, 1, kernel_size=(1, 1), stride=(1, 1), scale=0.0012004999443888664, zero_point=0)
  (relu): Identity()
  (quant): Quantize(scale=tensor([0.0345]), zero_point=tensor([55]), dtype=torch.quint8)
  (dequant): DeQuantize()
)

In [22]:
# 查看权重的数据类型：
model_int8.conv.weight().dtype

torch.qint8

In [23]:
# 可以看出此时权重的元素大小为 1 字节，而不是 FP32 的 4 字节：
model_int8.conv.weight().element_size()


1

In [24]:
# 运行模型，相关的计算将在 {data}torch.qint8 中发生。
res = model_int8(input_fp32)
res.dtype

量化前的类型： torch.quint8
量化中的类型： torch.quint8
量化后的类型： torch.float32


torch.float32